In [1]:
import pandas as pd
import timeit
import random
from collections import defaultdict
import csv
import math
import numpy as np
import sys
import os
import matplotlib as mplt
#from matplotlib import pylab
#import pylab 
import scipy
import scipy.stats as stats
import sklearn
import statsmodels as stmd
import pdb
import os
from datetime import datetime, timedelta
import pickle

global mypath
global dspath

mypath=os.getcwd()
dspath = mypath + '/tmp/'
logpath = mypath + '/log/'

#create workDir if not exis
os.system("mkdir -p " + dspath)
os.system("mkdir -p " + logpath)
#!ls -lrt {logpath}
print (mypath)
print (dspath)

/home/layla/instWork/python_job/layla_recommender
/home/layla/instWork/python_job/layla_recommender/tmp/


In [7]:
!ls -lrt {mypath}

total 14952
drwxrwxr-x 2 layla layla     4096 Mar 16 15:05 log
drwxrwxr-x 2 layla layla     4096 Mar 16 15:05 tmp
-rw-rw-r-- 1 layla layla      840 Mar 21 18:32 trivia.py
-rwxr-xr-x 1 layla layla    17794 Mar 21 18:32 recommender.py
-rw-rw-r-- 1 layla layla     2126 Mar 21 18:32 Pageview.py
-rw-rw-r-- 1 layla layla      392 Mar 21 18:32 keen_count.py
-rw-rw-r-- 1 layla layla     5136 Mar 21 18:39 keen.py
-rw-rw-r-- 1 layla layla     5136 Mar 21 18:42 keen_v1.py
-rw-rw-r-- 1 layla layla    14362 Mar 21 19:44 keen_layla.ipynb
-rw-rw-r-- 1 layla layla 15212965 Mar 21 19:52 keen4.csv
-rw-rw-r-- 1 layla layla    12657 Mar 21 20:01 Recommender_layla.ipynb


In [2]:
import time
import re
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error


In [8]:
t0 = time.clock()
print (t0)
answer_question_path = mypath + '/keen4.csv'
print (answer_question_path)

2.166423
/home/layla/instWork/python_job/layla_recommender/keen4.csv


In [30]:
df_raw = pd.read_csv(answer_question_path, sep=",", encoding = "ISO-8859-1") #saved as .csv but tab delimi
df_raw.to_pickle(dspath + '/keendata4_pickle')

print(df_raw.columns)
df_raw.head(100)

Index(['questionUUID', 'pageview', 'request.siteUUID', 'result',
       'question_type', 'question', 'site', 'engagement_rate'],
      dtype='object')


questionUUID  pageview  \
0   0239343e-44e4-4a7e-9625-e4553bd8e745     39894   
1   0b706994-587b-48e7-9a12-25090893abb9     39894   
2   0d8574ab-06af-473c-a89c-3d4b46419440     39894   
3   0f7ef8e1-b4f1-41ae-91f5-62d33bc39f33     39894   
4   180a7fee-8435-4352-88f4-fae103fc7a37     39894   
5   1b1e90e2-2655-4f7f-bd52-2ceae44dc3eb     39894   
6   27ef5d48-746e-431f-9d27-8059e22d3a85     39894   
7   2e09f390-44d1-45d9-b20c-45f69e878b47     39894   
8   2ff9c275-0ebd-40e7-b06c-f087a05e8ce8     39894   
9   30bacae0-d2ca-4fcf-b2d9-1cb131332319     39894   
10  32346a84-3a45-4599-8f0d-3c78f0078458     39894   
11  3388c1fe-63bd-4bd0-b036-ca07cbb91bd9     39894   
12  355afa93-90a5-46fb-a209-9c23014bfcea     39894   
13  42cb004d-81da-4f12-9b65-02af6a595a60     39894   
14  44ecfeeb-ff80-43b1-9c33-247018b9396b     39894   
15  460bd1aa-d54e-4350-b915-1c57f633d048     39894   
16  46e30742-6064-4ca2-ac7c-bb178a42e3dc     39894   
17  5cb5c9b9-dbec-4995-86c6-219f33af37be     39894   
18  601e303e-b131-4329-b031-c5f94e1a967a     39894   
19  7428cc13-5368-462b-9953-ed8307a50e52     39894   
20  7999ea74-8077-4481-8361-e02adba34ddf     39894   
21  84a72c8a-62a9-4c4c-8ec9-adec118c8dc3     39894   
22  9e06cf44-0711-4f6c-ad9e-1c0dedbfc74a     39894   
23  af1a9536-b537-4ae1-a425-96c90f714b68     39894   
24  b0229708-d953-4493-81e4-17d04e63203c     39894   
25  b34fe607-0455-4c3f-ad1f-9940f8488efb     39894   
26  ba17b3d5-ab0b-454b-a5ef-b542a7053630     39894   
27  bd4d340b-9332-43eb-878b-680c3b271671     39894   
28  c223214c-42d1-4ea9-8d01-2a3957712154     39894   
29  c847b514-17c9-48d4-8ef4-3ae0415e9c91     39894   
..                                   ...       ...   
70  0beef18d-e454-46be-94eb-d0aaf0ac8a1a   1361462   
71  0d4cdc00-0766-4d41-b0dc-65f6aa17823e   1361462   
72  0dff9501-34f2-4d7c-b7ea-ac3953f1aa2a   1361462   
73  0e42c671-50e1-469b-81a0-25146def7fbc   1361462   
74  0f376870-b585-4b66-a995-b42febb9dfe7   1361462   
75  104f6a48-66fe-43e9-bed0-709fd1e75e65   1361462   
76  1121825f-a09d-430e-9fff-a876a3d370d7   1361462   
77  11d1a74d-4108-4f7a-9d86-6685cd202819   1361462   
78  127ea901-3119-484f-b015-8d973d753c73   1361462   
79  12a735f8-0604-4c3a-941d-d3c17a3561f3   1361462   
80  12b59745-3d47-4a01-a2bc-e65a2855bd62   1361462   
81  12e15d83-09f7-422d-af16-3f58d6047048   1361462   
82  132f5c53-9dc0-45b8-9613-74d59f10100d   1361462   
83  137b297c-108f-438a-96cf-e379df711ef6   1361462   
84  13b734c7-b844-4829-9780-0117b0aa9a9d   1361462   
85  14011b5a-28e4-4dd2-8343-c0909cf8b742   1361462   
86  1413d78e-873c-474c-898e-58f9d0a87622   1361462   
87  142bbb48-1043-4d68-ab17-3d1552a37969   1361462   
88  14c09d57-a7c6-401c-9a83-0b537029932e   1361462   
89  14dbc3cf-2984-489a-a522-7ebbb50f51ef   1361462   
90  15259383-6f7d-418e-bf01-dadde0ab9cb1   1361462   
91  153107c9-3396-4616-93c2-3e2ebab19a08   1361462   
92  17229f31-111a-404e-943d-b5c57c485a01   1361462   
93  17318dda-ba8c-4c34-9a21-4c8cc31f6659   1361462   
94  1776cd2b-9d7e-4a74-8a3b-ab9d56cf4175   1361462   
95  17952ca0-81e0-4ab3-871d-eaeca0d3aeb7   1361462   
96  17d08939-2324-4027-b151-712223d336b1   1361462   
97  192b6be8-5318-4b0f-af4e-35c0b6017e40   1361462   
98  193c9ce9-de02-43f1-8de2-b50812889146   1361462   
99  1955aef0-5502-4492-b318-9946d5ef4afb   1361462   

                        request.siteUUID  result            question_type  \
0   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1                       TV   
1   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1                    Music   
2   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1                    Music   
3   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1  General American Sports   
4   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1                    Music   
5   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1                       TV   
6   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1                    Music   
7   00efda4b-a1dc-4dea-891d-fd4e4bfaa3f9       1               

In [12]:
answer_question_df = pd.read_pickle(dspath + '/keendata4_pickle')

In [13]:
answer_question_df = answer_question_df[answer_question_df.pageview > 100]

answer_question_subset_df = answer_question_df[['request.siteUUID', 'questionUUID', 'engagement_rate']]

In [24]:
answer_question_subset_df.shape

(73214, 3)

In [16]:
# Metrics
#========#### Mean Square Error for Collaborative Filtering ########
n_full_site = len(set(answer_question_subset_df['request.siteUUID']))
n_full_question = len(set(answer_question_subset_df['questionUUID']))


print (n_full_site)
print(n_full_question)

352
27270


In [21]:
# Content-based Algorithm
#================================================================================================================================================================

stopwords_list = stopwords.words('english') + stopwords.words('spanish')
# print (stopwords_list)
vectorizer = TfidfVectorizer(analyzer = 'word',
							ngram_range = (1, 3),
							min_df = 0.003,
							max_df = 0.8,
							max_features = 5000,
							stop_words = stopwords_list)

question_ids = answer_question_df['questionUUID'].tolist()


In [22]:
# Spanish Reading issue needs to be fixed
question_list = []
for question in answer_question_df['question'].tolist():
	try:
		question_list.append(str(question).encode('utf-8').strip())
	except:
		# try:
		# 	# print(re.sub(r'[^\w]', ' ', str(question)))
		question_list.append(re.sub(r'[^\w]', ' ', str(question)).encode('utf-8').strip())
		# except:
		# 	print(re.sub(r'[^\w]', ' ', str(question)))
		# 	pass
		# 	# print (question)

In [25]:
tfidf_matrix = vectorizer.fit_transform(question_list)
# print (dict(zip(vectorizer.get_feature_names(), vectorizer.idf_)))
tfidf_feature_names = vectorizer.get_feature_names()  #list of feature names

tfidf_feature_names[:100]

In [31]:
def get_question_profile(question_id):
	idx = question_ids.index(question_id)
	question_profile = tfidf_matrix[idx]
	return question_profile

def get_question_profiles(ids):
	question_profiles_list = []
	if type(ids) != str:
		question_profiles_list = [get_question_profile(x) for x in ids]
	else:
		question_profiles_list.append(get_question_profile(ids))
	question_profiles = scipy.sparse.vstack(question_profiles_list)
	return question_profiles

def build_sites_profile(site_id, interactions_indexed_df):
	interactions_site_df = interactions_indexed_df.loc[site_id]
	site_question_profiles = get_question_profiles(interactions_site_df['questionUUID'])
	site_question_strengths = np.array(interactions_site_df['engagement_rate']).reshape(-1,1)
	rows, cols = site_question_profiles.shape
	if rows > 1:
		site_question_strengths_weighted_avg = np.sum(site_question_profiles.multiply(site_question_strengths), axis=0) / np.sum(site_question_strengths)
	else:
		site_question_strengths_weighted_avg = site_question_profiles 
	site_profile_norm = sklearn.preprocessing.normalize(site_question_strengths_weighted_avg)
	return site_profile_norm

def build_sites_profiles(): 
	interactions_indexed_df = answer_question_subset_df.set_index('request.siteUUID')
	site_profiles = {}
	for site_id in interactions_indexed_df.index.unique():
		site_profiles[site_id] = build_sites_profile(site_id, interactions_indexed_df)
	return site_profiles


### Use above functions to build recommended questons for sites

In [40]:
# site_profiles = build_sites_profiles()  #run time 

In [34]:
type(site_profiles)

dict

In [59]:
list_key = list(site_profiles.keys())

In [45]:
list_val = list(site_profiles.values())[0:10]
print(list_val)

[array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.09193042, 0.        , 0.06910235, 0.08447823, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05717618, 0.        , 0.        ,
        0.        , 0.07930017, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.08467614, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08919743, 0.        , 0.        , 0.23954135,
        0.        , 0.        , 0.        , 0.07341782, 0.1820526 ,
        0.        , 0.        , 0.0692223 , 0.        , 0.        ,
        0.        , 0.        , 0.1491173 , 0.        , 0.        ,
        0.        , 0.        , 0.08614576, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.08857333, 0.        ,
        0.        , 0.        , 0.        , 0. 

In [64]:
sublist_key = list_key[25:30]
print('for those sites: ', sublist_key)

for those sites:  ['13156b2c-1453-4efe-94a7-ee59f7ad40fa', '1339d030-9b10-46e6-82f2-7469449d2ca5', '134cd2a0-7eab-4f89-ad56-f416cc5743c6', '136953cb-88f9-48b3-9417-8f28d8a3cf54', '14038036-d333-4247-a118-49e851066cde']


In [65]:
site_profiles[sublist_key[2]]

array([[1.52347593e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 4.55416249e-02, 7.62920449e-02, 1.03287507e-01,
        7.55508077e-02, 5.17631015e-02, 8.83703959e-02, 6.33241459e-02,
        6.41256393e-02, 1.38127391e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.69076110e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.22247225e-04, 8.93114228e-04, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 1.43781794e-02, 0.00000000e+00, 0.000000